In [1]:
from sqlalchemy import create_engine
eng = create_engine("duckdb:///:memory:")

In [2]:
from sqlalchemy import Column, Integer, Sequence, String, create_engine, JSON, ARRAY
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm.session import Session


In [3]:
Base = declarative_base()


class Patient(Base):
    __tablename__ = "patient"
    
    id = Column(String, primary_key=True) 
    name = Column(ARRAY(JSON))
    birthDate = Column(String) 
    gender = Column(String) 


In [4]:
from sqlalchemy.sql import text
with eng.connect() as con:
    con.execute(text("""CREATE TABLE patient AS SELECT * FROM  read_parquet('Patient.parquet/part-00000-d58f2ef8-cc43-48e1-9723-05966f96ca3f-c000.snappy.parquet')"""))
    con.commit()

In [5]:
with eng.connect() as con:
    for line in con.execute(text("""
select
 name[1].family as family,
 datediff('year', birthDate::date, current_date ) as age,
 gender as gender
from patient
limit 10
""")):
        print(line)

('Muller251', 34, 'male')
('Champlin946', 15, 'male')
('Stehr398', 52, 'female')
('Ebert178', 63, 'female')
('Carter549', 32, 'female')
('Kuvalis369', 45, 'male')
('Jacobs452', 60, 'female')
('Kihn564', 57, 'female')
('Dickens475', 12, 'female')
('Abshire638', 38, 'male')


In [6]:
from sqlalchemy import select, func

session = Session(bind=eng)

q = select(Patient.name[1]['family'])
#session.execute(q).all()
print(q)

SELECT (patient.name[:name_1])[:param_1] AS anon_1 
FROM patient
